In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3, preprocess_input
#IF THE INCEPTION IS TAKING TOO LONG.. TRY
#from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense,GlobalAveragePooling2D, Dropout, Flatten
from keras.models import Model
from keras.optimizers import Adam
import time

In [9]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

input_shape = 224

#YOU CAN REDUCE THE BATCH IF IT'S TAKING TOO LONG
train_gen = datagen.flow_from_directory('foodV2/train/', target_size=(input_shape,input_shape), batch_size=32,class_mode='categorical',shuffle=True, color_mode='rgb')
val_gen = datagen.flow_from_directory('foodV2/val/', target_size=(input_shape,input_shape), batch_size=32, class_mode='categorical',shuffle=True, color_mode='rgb')
test_gen = datagen.flow_from_directory('foodV2/test/', target_size=(input_shape,input_shape), batch_size=32, class_mode='categorical',shuffle=True, color_mode='rgb')

Found 2693 images belonging to 22 classes.
Found 2190 images belonging to 22 classes.
Found 2220 images belonging to 22 classes.


In [10]:
def foodModel(input_sh):
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(input_shape,input_shape,3), classes=1000)
    #USE VGG IF IT'S TAKING TOO LONG
    #base_model = VGG16(include_top=False, weights='imagenet', input_shape=(input_shape,input_shape,3), classes=1000)
    for layer in base_model.layers:
        #YOU CAN DECIDE IF YOU ARE GOING TO RETRAIN THE PRETRAINED MODEL, OR LEAVE IT AS IT IS WITH IT'S CURRENT WEIGHTS
        layer.trainable = False
    
    #YOU CAN REDUCE THE NUMBER OF DENSE LAYERS IF IT'S TAKING TOO LONG
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(22, activation="softmax")(x)
    
    model = Model(inputs=base_model.input,outputs=predictions)
    return model

In [11]:
%%time
foodModel = foodModel(input_shape)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
foodModel.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

CPU times: user 12.5 s, sys: 691 ms, total: 13.2 s
Wall time: 11.7 s


In [ ]:
step_size_train = train_gen.n//train_gen.batch_size
#REDUCE THE EPOCHS IF YOU WANT
foodModel.fit_generator(generator=train_gen,steps_per_epoch=step_size_train,epochs=100,verbose=1)

Epoch 1/100
34/84 [===========>..................] - ETA: 8:06 - loss: 9.9437 - accuracy: 0.0650 

In [ ]:
foodModel.save('nakedFoodModel.h5')

# THIS IS WHERE WE GET FUCKED! GOOD LUCK!

In [4]:
import tensorflow as tf

new_model= tf.keras.models.load_model(filepath="foodModelV1.h5")

converter = tf.lite.TFLiteConverter.from_saved_model(new_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

KeyError: 'weighted_metrics'